# Two-Soliton Orbit Demonstration

This notebook showcases a minimal PyUltraLight 2 setup with one massive soliton fixed at the origin and a lighter soliton on a circular orbit. We use PyUltraLight 2 helper utilities to draw the soliton profiles and visualise the equatorial density as an inline animation.

In [ ]:
# Patch dependencies to align with PyUltraLight 2 expectations and import the utilities we need.
import builtins
import contextlib
import numpy as np
import matplotlib.pyplot as plt
from matplotlib import animation
from IPython.display import HTML
import IPython.display as ipdisp
import IPython.core.display as core_disp
from scipy import integrate as SINT

# Ensure helpers that PyUltraLight 2 expects are available in this runtime.
if not hasattr(core_disp, "clear_output"):
    core_disp.clear_output = ipdisp.clear_output

if not hasattr(SINT, "simps"):
    SINT.simps = SINT.simpson


@contextlib.contextmanager
def _pyul_import_patch():
    """Temporarily satisfy PyUltraLight 2's interactive import prompts."""
    original_input = getattr(builtins, "_pyul_original_input", builtins.input)
    builtins._pyul_original_input = original_input
    builtins.input = lambda prompt='': '1e-22'
    try:
        yield
    finally:
        builtins.input = original_input


with _pyul_import_patch():
    import PyUltraLight2 as PyUL

%matplotlib inline
plt.style.use('dark_background')
plt.rcParams["figure.figsize"] = (6, 6)


In [ ]:
# Simulation parameters and helper functions.
resol = 64
length = 12.0
grid = np.linspace(-length / 2, length / 2, resol, endpoint=False)

big_mass = 12.0
small_mass = 3.0
orbit_radius = 2.5
num_frames = 48


def mass_to_alpha(mass):
    """Convert a soliton mass in code units to the matching profile scale parameter."""
    return (mass / 3.883) ** 2


def soliton_density(alpha, position):
    wave = PyUL.InitSolitonF(grid, position, resol, alpha)
    return np.abs(wave) ** 2


def density_slice(angle, alpha_small, big_density):
    position = [orbit_radius * np.cos(angle), orbit_radius * np.sin(angle), 0.0]
    small_density = soliton_density(alpha_small, position)
    return (big_density + small_density)[:, :, resol // 2]


alpha_big = mass_to_alpha(big_mass)
alpha_small = mass_to_alpha(small_mass)

# Build the central soliton once; we will reuse its density for every frame.
big_wave = PyUL.InitSolitonF(grid, [0.0, 0.0, 0.0], resol, alpha_big)
big_density = np.abs(big_wave) ** 2

# Use the built-in estimator to find the circular orbit velocity at the chosen radius.
enclosed_mass, orbital_speed = PyUL.DefaultSolitonOrbit(
    resol, length, '', big_mass, '', orbit_radius, '', ''
)
angular_frequency = orbital_speed / orbit_radius
orbital_period = 2 * np.pi / angular_frequency

angles = np.linspace(0.0, 2.0 * np.pi, num_frames, endpoint=False)
frame_times = angles / angular_frequency
frame_duration = frame_times[1] - frame_times[0] if len(frame_times) > 1 else orbital_period

# Pre-compute the 2-D density slices used by the animation.
density_slices = np.stack([
    density_slice(angle, alpha_small, big_density) for angle in angles
])

extent = [grid[0], grid[0] + length, grid[0], grid[0] + length]
density_min = density_slices.min()
density_max = density_slices.max()

print(f"Enclosed mass within {orbit_radius:.2f} code units: {enclosed_mass:.3f}")
print(f"Circular velocity at that radius: {orbital_speed:.3f} code units")
print(f"Orbital period: {orbital_period:.3f} code time units")


In [ ]:
# Build and display the animation of the orbital plane density.
fig, ax = plt.subplots()
ax.set_aspect('equal')
ax.set_xlim(extent[0], extent[1])
ax.set_ylim(extent[2], extent[3])

orbit_theta = np.linspace(0, 2 * np.pi, 256)
orbit_path, = ax.plot(
    orbit_radius * np.cos(orbit_theta),
    orbit_radius * np.sin(orbit_theta),
    color='white', linestyle='--', linewidth=0.8, label='Orbit path'
)
ax.scatter([0.0], [0.0], color='gold', s=40, marker='*', label='Central soliton')

norm = plt.Normalize(vmin=density_min, vmax=density_max)
im = ax.imshow(
    density_slices[0],
    cmap='magma',
    origin='lower',
    extent=extent,
    norm=norm
)
ax.set_xlabel('x (code units)')
ax.set_ylabel('y (code units)')
ax.set_title('Equatorial density – frame 1 (t = 0.00)')

cbar = fig.colorbar(im, ax=ax, fraction=0.046, pad=0.04)
cbar.set_label('Density (code units)')

orbit_marker, = ax.plot([], [], marker='o', color='cyan', markersize=5, linestyle='None', label='Orbital soliton')
ax.legend(loc='upper right')


def update(frame):
    im.set_data(density_slices[frame])
    angle = angles[frame]
    orbit_marker.set_data(
        [orbit_radius * np.cos(angle)],
        [orbit_radius * np.sin(angle)],
    )
    current_time = frame_times[frame]
    ax.set_title(f'Equatorial density – frame {frame + 1} (t = {current_time:.2f})')
    return im, orbit_marker


anim = animation.FuncAnimation(
    fig,
    update,
    frames=num_frames,
    interval=1000 * frame_duration,
    blit=False,
    repeat=True
)
plt.close(fig)
HTML(anim.to_jshtml())
